In [1]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError
import noisereduce

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024 * 2             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

In [2]:
# create matplotlib figure and axes
fig, ax = plt.subplots(figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

ax.set_xlim(20, RATE / 2)
ax.set_ylim(0, 3)

window = 0.5 * (1 - np.cos(np.linspace(0, 2*np.pi, CHUNK, False)))

x = np.linspace(0, RATE, CHUNK)

noise_reduced, = ax.semilogx(x, np.random.rand(CHUNK), '-', lw=2)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    data = stream.read(CHUNK)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    ffted = np.fft.rfft(data_np * window)
    
    magnitude = np.abs(ffted)
    
    magnitude_db = 20 * np.log10(magnitude)
    
    noise_reduced.set_ydata(magnitude_db)
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

stream started


ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (2048,) and arg 1 with shape (1025,).

Exception in Tkinter callback
Traceback (most recent call last):
  File "e:\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "e:\anaconda3\lib\tkinter\__init__.py", line 839, in callit
    func(*args)
  File "e:\anaconda3\lib\site-packages\matplotlib\backends\_backend_tk.py", line 271, in idle_draw
    self.draw()
  File "e:\anaconda3\lib\site-packages\matplotlib\backends\backend_tkagg.py", line 10, in draw
    super().draw()
  File "e:\anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 400, in draw
    self.figure.draw(self.renderer)
  File "e:\anaconda3\lib\site-packages\matplotlib\artist.py", line 95, in draw_wrapper
    result = draw(artist, renderer, *args, **kwargs)
  File "e:\anaconda3\lib\site-packages\matplotlib\artist.py", line 72, in draw_wrapper
    return draw(artist, renderer)
  File "e:\anaconda3\lib\site-packages\matplotlib\figure.py", line 3125, in draw
    mimage._draw_list_compositing_images(
  File "e:\